In [364]:
import json
train_json = 'train2_swj.json'
with open(train_json, 'r') as f:
    data = json.load(f)
    info = data['info']
    licenses = data['licenses']
    images = data['images']
    categories = data['categories']
    annotations = data['annotations']

## 적용할 Augmentation

In [365]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=30, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=(-0.3, 0.3), contrast_limit=(-0.3, 0.3), p=0.5),
], bbox_params=A.BboxParams(format='coco', label_fields=['category_id']))

## 예시 이미지 출력

In [ ]:
battery_image_id = [ann['image_id'] for ann in annotations if ann['category_id']==8]
print(battery_image_id)
print(len(battery_image_id))
train_ann_num = len(battery_image_id) # val에 사용
battery_image_id = list(set(battery_image_id))
print(battery_image_id)
print(len(battery_image_id))
train_img_num = len(battery_image_id) # val에 사용

In [ ]:
import cv2
import numpy as np
from pycocotools.coco import COCO
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
from PIL import Image

# 예시 이미지 출력해보기
coco = COCO(train_json)

n = battery_image_id[24]
image_id = coco.getImgIds(imgIds=n)
image_info = coco.loadImgs(image_id)[0]

box = [x['bbox'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]
cat = [coco.loadCats(x['category_id'])[0]['name'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]

img = cv2.imread(image_info['file_name'])
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
img /= 255.0

classes = ["General trash", "Paper", "Paper pack", "Metal", "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]
LABELCOLORS = ['red', 'orange', 'yellow', 'greenyellow', 'green', 'turquoise', 'blue', 'indigo', 'purple', 'black']

transformed = transform(image=img, bboxes=box, category_id=cat)
transformed_img = transformed['image']
transformed_bboxes = transformed['bboxes']

fig, ax = plt.subplots(1, 2, figsize=(12, 6))

ax[0].set_xticks([])
ax[0].set_yticks([])

# plot original image
ax[0].imshow(img)
for (x, y, w, h), c in zip(box, cat):
    label_color = LABELCOLORS[classes.index(c)]
    ax[0].add_patch(patches.Rectangle((x, y), w, h, edgecolor=label_color, linewidth=1, fill=False))
    ax[0].text(x, y-5, c, fontsize=9, color=label_color)
ax[0].set_title(f'Original Image {n}')

ax[1].set_xticks([])
ax[1].set_yticks([])

ax[1].imshow(np.transpose(transformed_img, (0, 1, 2)))
for (x, y, w, h), c in zip(transformed_bboxes, cat):
    label_color = LABELCOLORS[classes.index(c)]
    ax[1].add_patch(patches.Rectangle((x, y), w, h, edgecolor=label_color, linewidth=1, fill=False))
    ax[1].text(x, y-5, c, fontsize=9, color=label_color)
ax[1].set_title(f'Transformed Image {n}')

plt.show()

## Train dataset Up-sampling

In [368]:
classes = ["General trash", "Paper", "Paper pack", "Metal", "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]
LABELCOLORS = ['red', 'orange', 'yellow', 'greenyellow', 'green', 'turquoise', 'blue', 'indigo', 'purple', 'black']

battery_image_id = [ann['image_id'] for ann in annotations if ann['category_id']==8]
battery_image_id = list(set(battery_image_id))

# num은 train.json 기준
image_num = 4883
annotation_num = 23144

coco = COCO(train_json)
for n in range(5):  # 몇 배 up-sampling 시킬지 (5배)
    print('======== ', n ,' =========')
    for bii in battery_image_id: # battery가 포함된 image_id 들
        print('current image_id :', bii)
        image_id = coco.getImgIds(imgIds=bii)
        image_info = coco.loadImgs(image_id)[0]

        box = [x['bbox'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]
        cat = [coco.loadCats(x['category_id'])[0]['name'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]

        img = cv2.imread(image_info['file_name'])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img /= 255.0

        transformed = transform(image=img, bboxes=box, category_id=cat)
        transformed_img = transformed['image']
        transformed_bboxes = transformed['bboxes']

        # 이미지 저장
        image_num += 1
        images.append({
            'width': transformed_img.shape[0],
            'height': transformed_img.shape[1],
            'file_name': 'battery_aug/'+str(image_num)+'.jpg',
            'license': 0,
            'flickr_url': None,
            'coco_url': None,
            'date_captured': None,
            'id': image_num
        })
        # 변환된 이미지 파일 저장
        transformed_img = Image.fromarray((transformed_img * 255).astype(np.uint8))
        transformed_img.save("battery_aug/"+str(image_num)+'.jpg')

        # annotations 저장
        for (x, y, w, h), c in zip(transformed_bboxes, cat):
            annotation_num += 1
            annotations.append({
                'image_id': image_num,
                'category_id': classes.index(c),
                'area': round(w*h, 2),
                'bbox': [round(x, 1), round(y, 1), round(w, 1), round(h, 1)],
                'iscrowd': 0,
                'id': annotation_num})

# json 파일 저장
train_battery_json = 'train2_swj_battery.json'
with open(train_battery_json, 'w') as f:
    json.dump({ 'info': info, 'licenses': licenses, 'images': images, 
            'annotations': annotations, 'categories': categories}, f)

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
========  0  =========
current image_id : 643
current image_id : 4867
current image_id : 1157
current image_id : 2187
current image_id : 2582
current image_id : 1175
current image_id : 1944
current image_id : 3996
current image_id : 3997
current image_id : 3360
current image_id : 1825
current image_id : 4385
current image_id : 291
current image_id : 675
current image_id : 555
current image_id : 557
current image_id : 4143
current image_id : 3122
current image_id : 821
current image_id : 949
current image_id : 2871
current image_id : 1209
current image_id : 4285
current image_id : 319
current image_id : 449
current image_id : 1861
current image_id : 2763
current image_id : 204
current image_id : 4555
current image_id : 1492
current image_id : 4185
current image_id : 3930
current image_id : 2139
current image_id : 92
current image_id : 3292
current image_id : 3549
current image_id : 879
current image_id : 

## Validation dataset Up-sampling

In [369]:
import json
val_json = 'val2_swj.json'
with open(val_json, 'r') as f:
    data = json.load(f)
    info = data['info']
    licenses = data['licenses']
    images = data['images']
    categories = data['categories']
    annotations = data['annotations']

In [ ]:
battery_image_id = [ann['image_id'] for ann in annotations if ann['category_id']==8]
print(battery_image_id)
print(len(battery_image_id))
battery_image_id = list(set(battery_image_id))
print(battery_image_id)
print(len(battery_image_id))

In [371]:
classes = ["General trash", "Paper", "Paper pack", "Metal", "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]
LABELCOLORS = ['red', 'orange', 'yellow', 'greenyellow', 'green', 'turquoise', 'blue', 'indigo', 'purple', 'black']

battery_image_id = [ann['image_id'] for ann in annotations if ann['category_id']==8]
battery_image_id = list(set(battery_image_id))

# num은 train.json 기준
image_num = 4883+train_img_num*5
annotation_num = 23144+train_ann_num*5

coco = COCO(val_json)
for n in range(5):  # 몇 배 up-sampling 시킬지 (5배)
    print('======== ', n ,' =========')
    for bii in battery_image_id: # battery가 포함된 image_id 들
        print('current image_id :', bii)
        image_id = coco.getImgIds(imgIds=bii)
        image_info = coco.loadImgs(image_id)[0]

        box = [x['bbox'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]
        cat = [coco.loadCats(x['category_id'])[0]['name'] for x in coco.loadAnns(coco.getAnnIds(imgIds=image_info['id']))]

        img = cv2.imread(image_info['file_name'])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32)
        img /= 255.0

        transformed = transform(image=img, bboxes=box, category_id=cat)
        transformed_img = transformed['image']
        transformed_bboxes = transformed['bboxes']

        # 이미지 저장
        image_num += 1
        images.append({
            'width': transformed_img.shape[0],
            'height': transformed_img.shape[1],
            'file_name': 'battery_aug/'+str(image_num)+'.jpg',
            'license': 0,
            'flickr_url': None,
            'coco_url': None,
            'date_captured': None,
            'id': image_num
        })
        # 변환된 이미지 파일 저장
        transformed_img = Image.fromarray((transformed_img * 255).astype(np.uint8))
        transformed_img.save("battery_aug/"+str(image_num)+'.jpg')

        # annotations 저장
        for (x, y, w, h), c in zip(transformed_bboxes, cat):
            annotation_num += 1
            annotations.append({
                'image_id': image_num,
                'category_id': classes.index(c),
                'area': round(w*h, 2),
                'bbox': [round(x, 1), round(y, 1), round(w, 1), round(h, 1)],
                'iscrowd': 0,
                'id': annotation_num})

# json 파일 저장
train_battery_json = 'val2_swj_battery.json'
with open(train_battery_json, 'w') as f:
    json.dump({ 'info': info, 'licenses': licenses, 'images': images, 
            'annotations': annotations, 'categories': categories}, f)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
========  0  =========
current image_id : 1154
current image_id : 3886
current image_id : 1991
========  1  =========
current image_id : 1154
current image_id : 3886
current image_id : 1991
========  2  =========
current image_id : 1154
current image_id : 3886
current image_id : 1991
========  3  =========
current image_id : 1154
current image_id : 3886
current image_id : 1991
========  4  =========
current image_id : 1154
current image_id : 3886
current image_id : 1991
